# Recommender Systems 2018/19

### Practice session on BPR-MF


## Recap on BPR
S.Rendle et al. BPR: Bayesian Personalized Ranking from Implicit Feedback. UAI2009

The usual approach for item recommenders is to predict a personalized score $\hat{x}_{ui}$ for an item that reflects the preference of the user for the item. Then the items are ranked by sorting them according to that score.

Machine learning approaches are tipically fit by using observed items as a positive sample and missing ones for the negative class. A perfect model would thus be useless, as it would classify as negative (non-interesting) all the items that were non-observed at training time. The only reason why such methods work is regularization.

BPR use a different approach. The training dataset is composed by triplets $(u,i,j)$ representing that user u is assumed to prefer i over j. For an implicit dataset this means that u observed i but not j:
$$D_S := \{(u,i,j) \mid i \in I_u^+ \wedge j \in I \setminus I_u^+\}$$

### BPR-OPT
A machine learning model can be represented by a parameter vector $\Theta$ which is found at fitting time. BPR wants to find the parameter vector that is most probable given the desired, but latent, preference structure $>_u$:
$$p(\Theta \mid >_u) \propto p(>_u \mid \Theta)p(\Theta) $$
$$\prod_{u\in U} p(>_u \mid \Theta) = \dots = \prod_{(u,i,j) \in D_S} p(i >_u j \mid \Theta) $$

The probability that a user really prefers item $i$ to item $j$ is defined as:
$$ p(i >_u j \mid \Theta) := \sigma(\hat{x}_{uij}(\Theta)) $$
Where $\sigma$ represent the logistic sigmoid and $\hat{x}_{uij}(\Theta)$ is an arbitrary real-valued function of $\Theta$ (the output of your arbitrary model).


To complete the Bayesian setting, we define a prior density for the parameters:
$$p(\Theta) \sim N(0, \Sigma_\Theta)$$
And we can now formulate the maximum posterior estimator:
$$BPR-OPT := \log p(\Theta \mid >_u) $$
$$ = \log p(>_u \mid \Theta) p(\Theta) $$
$$ = \log \prod_{(u,i,j) \in D_S} \sigma(\hat{x}_{uij})p(\Theta) $$
$$ = \sum_{(u,i,j) \in D_S} \log \sigma(\hat{x}_{uij}) + \log p(\Theta) $$
$$ = \sum_{(u,i,j) \in D_S} \log \sigma(\hat{x}_{uij}) - \lambda_\Theta ||\Theta||^2 $$

Where $\lambda_\Theta$ are model specific regularization parameters.

### BPR learning algorithm
Once obtained the log-likelihood, we need to maximize it in order to find our obtimal $\Theta$. As the crierion is differentiable, gradient descent algorithms are an obvious choiche for maximization.


The basic version of gradient descent consists in evaluating the gradient using all the available samples and then perform a single update. The problem with this is, in our case, that our training dataset is very skewed. Suppose an item i is very popular. Then we habe many terms of the form $\hat{x}_{uij}$ in the loss because for many users u the item i is compared against all negative items j.

The other popular approach is stochastic gradient descent, where for each training sample an update is performed. This is a better approach, but the order in which the samples are traversed is crucial. To solve this issue BPR uses a stochastic gradient descent algorithm that choses the triples randomly.

The gradient of BPR-OPT with respect to the model parameters is: 
$$\frac{\partial BPR-OPT}{\partial \Theta} = \sum_{(u,i,j) \in D_S} \frac{\partial}{\partial \Theta} \log \sigma (\hat{x}_{uij}) - \lambda_\Theta \frac{\partial}{\partial\Theta} || \Theta ||^2$$
$$ =  \sum_{(u,i,j) \in D_S} \frac{-e^{-\hat{x}_{uij}}}{1+e^{-\hat{x}_{uij}}} \frac{\partial}{\partial \Theta}\hat{x}_{uij} - \lambda_\Theta \Theta $$

### BPR-MF

In order to practically apply this learning schema to an existing algorithm, we first split the real valued preference term: $\hat{x}_{uij} := \hat{x}_{ui} − \hat{x}_{uj}$. And now we can apply any standard collaborative filtering model that predicts $\hat{x}_{ui}$.

The problem of predicting $\hat{x}_{ui}$ can be seen as the task of estimating a matrix $X:U×I$. With matrix factorization teh target matrix $X$ is approximated by the matrix product of two low-rank matrices $W:|U|\times k$ and $H:|I|\times k$:
$$X := WH^t$$
The prediction formula can also be written as:
$$\hat{x}_{ui} = \langle w_u,h_i \rangle = \sum_{f=1}^k w_{uf} \cdot h_{if}$$
Besides the dot product ⟨⋅,⋅⟩, in general any kernel can be used.

We can now specify the derivatives:
$$ \frac{\partial}{\partial \theta} \hat{x}_{uij} = \begin{cases}
(h_{if} - h_{jf}) \text{ if } \theta=w_{uf}, \\
w_{uf} \text{ if } \theta = h_{if}, \\
-w_{uf} \text{ if } \theta = h_{jf}, \\
0 \text{ else }
\end{cases} $$

Which basically means: user $u$ prefer $i$ over $j$, let's do the following:
- Increase the relevance (according to $u$) of features belonging to $i$ but not to $j$ and vice-versa
- Increase the relevance of features assigned to $i$
- Decrease the relevance of features assigned to $j$

We're now ready to look at some code!

In [1]:
from urllib.request import urlretrieve
import zipfile

# skip the download
#urlretrieve ("http://files.grouplens.org/datasets/movielens/ml-10m.zip", "data/Movielens_10M/movielens_10m.zip")
dataFile = zipfile.ZipFile("data/Movielens_10M/movielens_10m.zip")
URM_path = dataFile.extract("ml-10M100K/ratings.dat", path = "data/Movielens_10M")
URM_file = open(URM_path, 'r')


def rowSplit (rowString):
    
    split = rowString.split("::")
    split[3] = split[3].replace("\n","")
    
    split[0] = int(split[0])
    split[1] = int(split[1])
    split[2] = float(split[2])
    split[3] = int(split[3])
    
    result = tuple(split)
    
    return result


URM_file.seek(0)
URM_tuples = []

for line in URM_file:
   URM_tuples.append(rowSplit (line))

userList, itemList, ratingList, timestampList = zip(*URM_tuples)

userList = list(userList)
itemList = list(itemList)
ratingList = list(ratingList)
timestampList = list(timestampList)

import scipy.sparse as sps

URM_all = sps.coo_matrix((ratingList, (userList, itemList)))
URM_all = URM_all.tocsr()



from Notebooks_utils.data_splitter import train_test_holdout


URM_train, URM_test = train_test_holdout(URM_all, train_perc = 0.8)

### MF Computing prediction

### In a MF model you have two matrices, one with a row per user and the other with a column per item. The other dimension, columns for the first one and rows for the second one is called latent factors

In [2]:
num_factors = 10

n_users, n_items = URM_train.shape

In [4]:
import numpy as np

user_factors = np.random.random((n_users, num_factors))

item_factors = np.random.random((n_items, num_factors))

user_factors

array([[0.73517132, 0.24611999, 0.57612737, ..., 0.11108283, 0.20387197,
        0.85977337],
       [0.79558673, 0.90064645, 0.70967288, ..., 0.0721837 , 0.28162736,
        0.15360077],
       [0.60193939, 0.73445386, 0.15425944, ..., 0.9894053 , 0.29166679,
        0.10166429],
       ...,
       [0.37568984, 0.27687165, 0.14860683, ..., 0.30045171, 0.18268336,
        0.60046981],
       [0.99410918, 0.66675883, 0.08897947, ..., 0.09980786, 0.1293671 ,
        0.05086965],
       [0.86258766, 0.32414275, 0.35135127, ..., 0.21536246, 0.98314574,
        0.51128304]])

### To compute the prediction we have to muliply the user factors to the item factors

In [5]:
item_index = 15
user_index = 42

prediction = np.dot(user_factors[user_index,:], item_factors[item_index,:])

print("Prediction is {:.2f}".format(prediction))

Prediction is 0.90


# Train a MF MSE model

### Use SGD as we saw for SLIM

In [6]:
test_data = 5
learning_rate = 1e-2
regularization = 1e-3

gradient = test_data - prediction

print("Prediction error is {:.2f}".format(gradient))

Prediction error is 4.10


In [7]:
# Copy original value to avoid messing up the updates
H_i = item_factors[item_index,:]
W_u = user_factors[user_index,:]

user_factors[user_index,:] += learning_rate * (gradient * H_i - regularization * W_u)
item_factors[item_index,:] += learning_rate * (gradient * W_u - regularization * H_i)


In [8]:
prediction = np.dot(user_factors[user_index,:], item_factors[item_index,:])

print("Prediction after the update is {:.2f}".format(prediction))
print("Prediction error is {:.2f}".format(test_data - prediction))

Prediction after the update is 1.06
Prediction error is 3.94


### WARNING: Initialization must be done with random non-zero values ... otherwise

In [9]:
user_factors = np.zeros((n_users, num_factors))

item_factors = np.zeros((n_items, num_factors))

In [10]:
prediction = np.dot(user_factors[user_index,:], item_factors[item_index,:])

print("Prediction is {:.2f}".format(prediction))

gradient = test_data - prediction

print("Prediction error is {:.2f}".format(gradient))

Prediction is 0.00
Prediction error is 5.00


In [11]:
H_i = item_factors[item_index,:]
W_u = user_factors[user_index,:]

user_factors[user_index,:] += learning_rate * (gradient * H_i - regularization * W_u)
item_factors[item_index,:] += learning_rate * (gradient * W_u - regularization * H_i)


In [12]:
prediction = np.dot(user_factors[user_index,:], item_factors[item_index,:])

print("Prediction after the update is {:.2f}".format(prediction))
print("Prediction error is {:.2f}".format(test_data - prediction))

Prediction after the update is 0.00
Prediction error is 5.00


### Since the updates multiply the gradient and the latent factors, if those are zero the SGD will never be able to move from that point

# Train a MF BPR model

## The basics are the same, except for how we compute the gradient, we have to sample a triplet

In [13]:
URM_mask = URM_train.copy()
URM_mask.data[URM_mask.data <= 3] = 0

URM_mask.eliminate_zeros()

# Extract users having at least one interaction to choose from
eligibleUsers = []

for user_id in range(n_users):

    start_pos = URM_mask.indptr[user_id]
    end_pos = URM_mask.indptr[user_id+1]

    if len(URM_mask.indices[start_pos:end_pos]) > 0:
        eligibleUsers.append(user_id)
                
                

def sampleTriplet():
    
    # By randomly selecting a user in this way we could end up 
    # with a user with no interactions
    #user_id = np.random.randint(0, n_users)
    
    user_id = np.random.choice(eligibleUsers)
    
    # Get user seen items and choose one
    userSeenItems = URM_mask[user_id,:].indices
    pos_item_id = np.random.choice(userSeenItems)

    negItemSelected = False

    # It's faster to just try again then to build a mapping of the non-seen items
    while (not negItemSelected):
        neg_item_id = np.random.randint(0, n_items)

        if (neg_item_id not in userSeenItems):
            
            negItemSelected = True

    return user_id, pos_item_id, neg_item_id


In [14]:
for _ in range(10):
    print(sampleTriplet())

(38797, 1321, 5722)
(39156, 3481, 38129)
(69352, 1234, 23469)
(34004, 1198, 21578)
(26917, 1625, 23110)
(17111, 1208, 62312)
(10435, 25, 39704)
(52173, 1221, 44183)
(23217, 753, 11713)
(12801, 60074, 24449)


In [15]:
user_factors = np.random.random((n_users, num_factors))
item_factors = np.random.random((n_items, num_factors))

In [16]:
user_id, positive_item, negative_item = sampleTriplet()

print(user_id, positive_item, negative_item)

66319 996 36479


In [17]:
x_uij = np.dot(user_factors[user_id, :], (item_factors[positive_item,:] - item_factors[negative_item,:]))

x_uij

-0.9154786598249658

In [18]:
sigmoid_item = 1 / (1 + np.exp(x_uij))

sigmoid_item

0.7141199564607723

### When using BPR we have to update three components, the user factors and the item factors of both the positive and negative item

In [20]:

H_i = item_factors[positive_item,:]
H_j = item_factors[negative_item,:]
W_u = user_factors[user_id,:]


user_factors[user_index,:] += learning_rate * (sigmoid_item * ( H_i - H_j ) - regularization * W_u)
item_factors[positive_item,:] += learning_rate * (sigmoid_item * ( W_u ) - regularization * H_i)
item_factors[negative_item,:] += learning_rate * (sigmoid_item * (-W_u ) - regularization * H_j)


In [21]:
x_uij = np.dot(user_factors[user_id, :], (item_factors[positive_item,:] - item_factors[negative_item,:]))

x_uij

-0.7523443955087056

In [22]:
## How to rank items with MF ?

## Compute the prediction for all items and rank them

item_scores = np.dot(user_factors[user_index,:], item_factors.T)
item_scores

array([3.871955  , 3.0048805 , 2.33520388, ..., 4.06992949, 2.3217676 ,
       2.77732822])

In [23]:
item_scores.shape

(65134,)

## Early stopping, how to used and when it is needed

### Problem, how many epochs? 5, 10, 150, 2487 ?

### We could try different values in increasing order: 5, 10, 15, 20, 25...
### However, in this way we would train up to a point, test and then discard the model, to re-train it again up to that same point and then some more... not a good idea.

### Early stopping! 
* Train the model up to a certain number of epochs, say 5
* Compute the recommendation quality on the validation set
* Train for other 5 epochs
* Compute the recommendation quality on the validation set AND compare it with the previous one. If better, then we have another best model, if not, go ahead...
* Repeat until you have either reached the max number of epoch you want to allow (e.g., 300) or a certain number of contiguous validation seps have not updated te best model

### Advantages:
* Easy to implement, we already have all that is required, a train function, a predictor function and an evaluator
* MUCH faster than retraining everything from the beginning
* Often allows to reach even better solutions

### Challenges:
* The evaluation step may be very slow compared to the time it takes to re-train the model

# Train a PureSVD model

### As opposed to the previous ones, PureSVD relies on the SVD decomposition of the URM, which is an easily available function

In [25]:
from sklearn.utils.extmath import randomized_svd

# Other SVDs are also available, like from sklearn.decomposition import TruncatedSVD

In [26]:
U, Sigma, VT = randomized_svd(URM_train,
              n_components=num_factors,
              #n_iter=5,
              random_state=None)

In [27]:
U

array([[ 1.33930046e-21,  4.50017600e-16, -3.45360900e-17, ...,
        -4.99420423e-15,  4.14740490e-15,  3.42800490e-15],
       [ 8.39714525e-04, -3.33026143e-03, -7.93229895e-04, ...,
        -1.04935468e-03, -1.45777986e-03,  1.66068353e-04],
       [ 7.15527244e-04, -1.46693150e-03, -8.39343520e-05, ...,
         4.38058190e-03,  2.54407548e-03, -5.60247703e-04],
       ...,
       [ 3.06478715e-03,  2.02402932e-03,  5.63169174e-03, ...,
        -7.41287413e-04,  1.68624957e-03, -3.26572191e-03],
       [ 1.36684444e-03, -5.06991919e-03,  8.57030322e-04, ...,
         3.49433414e-04, -1.13290000e-03, -2.05880644e-03],
       [ 1.47303160e-03, -9.24507444e-04, -6.93847190e-04, ...,
         2.63330612e-03,  4.54485374e-04, -5.63922632e-03]])

In [28]:
U.shape

(71568, 10)

In [29]:
Sigma

array([4272.51629214, 1781.7315434 , 1530.25816798, 1225.44173369,
       1183.32515771, 1013.97053936,  960.51194136,  908.69494193,
        842.86306518,  744.7344711 ])

In [30]:
Sigma.shape

(10,)

In [31]:
VT

array([[ 3.07928040e-22,  7.97868590e-02,  3.47401413e-02, ...,
         0.00000000e+00,  0.00000000e+00,  3.52033205e-05],
       [ 2.55227025e-16, -4.72463012e-02, -5.00928278e-02, ...,
        -0.00000000e+00, -0.00000000e+00,  5.59774136e-05],
       [ 2.33160085e-16, -1.22096349e-02, -2.15907865e-02, ...,
        -0.00000000e+00, -0.00000000e+00,  2.23214501e-05],
       ...,
       [ 3.53784390e-16,  1.42608193e-01,  2.41202623e-02, ...,
         0.00000000e+00,  0.00000000e+00,  1.06606346e-04],
       [-1.94658263e-16, -4.38327196e-02, -2.67418692e-02, ...,
         0.00000000e+00,  0.00000000e+00,  8.89050376e-05],
       [ 1.22478906e-16,  5.92158818e-03, -1.30203008e-03, ...,
        -0.00000000e+00, -0.00000000e+00, -5.30051871e-05]])

In [32]:
VT.shape

(10, 65134)

### Truncating the number of singular values introduces an approximation which allows to fill the missing urm entries

### Computing a prediction

In [33]:
# Store an intermediate pre-multiplied matrix

s_Vt = sps.diags(Sigma)*VT

In [34]:
prediction = U[user_index, :].dot(s_Vt[:,item_index])

print("Prediction is {:.2f}".format(prediction))

Prediction is 0.04


In [35]:
item_scores = U[user_index, :].dot(s_Vt)
item_scores

array([-5.08919251e-16,  6.45763385e-01,  4.67062293e-01, ...,
        0.00000000e+00,  0.00000000e+00,  3.34097614e-04])

In [36]:
item_scores.shape

(65134,)